In [2]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [4]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 133 kB 15.1 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [11]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape = (28, 28)))

    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate), 
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                  metrics = ['accuracy'])
    
    return model

In [12]:
tuner = kt.Hyperband(model_builder, 
                     objective ='val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name  = 'intro_to_kt')

In [13]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [14]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test) , callbacks = [ClearTrainingOutput()])

Trial 30 Complete [00h 01m 05s]
val_accuracy: 0.8495000004768372

Best val_accuracy So Far: 0.8859000205993652
Total elapsed time: 00h 18m 29s
INFO:tensorflow:Oracle triggered exit


In [15]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [16]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



In [17]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4815 - accuracy: 0.8293 - val_loss: 0.4375 - val_accuracy: 0.8405
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3634 - accuracy: 0.8683 - val_loss: 0.3778 - val_accuracy: 0.8624
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3260 - accuracy: 0.8796 - val_loss: 0.3765 - val_accuracy: 0.8641
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3012 - accuracy: 0.8893 - val_loss: 0.3736 - val_accuracy: 0.8636
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2849 - accuracy: 0.8945 - val_loss: 0.3524 - val_accuracy: 0.8723
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2703 - accuracy: 0.8994 - val_loss: 0.3295 - val_accuracy: 0.8824
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2565 - accuracy: 0.9032 - val_loss: 0.3413 - val_accuracy: